In [2]:
with open("test_files.txt", 'r') as file:
    lines = file.readlines()

parsed_data = [(int(line.split()[0]), line.split()[1]) for line in lines]

In [3]:
start_index = None
for i, (file, category) in enumerate(parsed_data):
    if category == 'Switch' and file == 100911:
        start_index = i
        break


In [ ]:
if start_index is None:
    print("Start index not found")
else:
    # Process the data starting from the found index
    io = IO()
    for file, category in parsed_data[start_index:]:
        xyz, rgb = normalize_pc(f"data/{category}/{file}/pc.ply", f'examples/zeroshot_{category}', io, device) #read Point cloud and rgb in the format n,3
        img_dir, pc_idx, screen_coords = render_pc(xyz, rgb, f'examples/zeroshot_{category}', device, file)
        
        np.savez(f'examples/zeroshot_{category}/rendered_img_test/{file}/pcidx_screencoords.npz', 
                 pc_idx=pc_idx, 
                 screen_coords=screen_coords)

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [ ]:

results = {}
for category in categories:
    filtered_data = [f"data/{category}/{item[0]}" for item in parsed_data if item[1] == category]
    results[category] = filtered_data[:num_files]


In [2]:
!python testIOU.py

[nltk_data] Downloading package punkt to
[nltk_data]     /rhome/kamburoglu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /rhome/kamburoglu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
final text_encoder_type: bert-base-uncased
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model)

In [6]:
import sys
sys.path.insert(0,'../GroundingDINO')
sys.path.insert(0,'../SAM')
import os
import torch
import json
from pytorch3d.io import IO
import numpy as np
from src.utils import normalize_pc,save_colored_pc
from src.render_pc import render_pc
from src.gen_superpoint import gen_superpoint
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2
from torchvision.ops import nms
import json
from segment_anything import sam_model_registry, SamPredictor
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import distinctipy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def yolobbox2bbox(yolobox):
    x = yolobox[:,0]
    y = yolobox[:,1]
    w = yolobox[:,2]
    h = yolobox[:,3]
    xyxy = np.zeros_like(yolobox)
    xyxy[:,0] = x-w/2
    xyxy[:,1] = y-h/2
    xyxy[:,2] = x+w/2
    xyxy[:,3] = y+h/2
    return xyxy

In [5]:
def check_pc_within_bbox(x1, y1, x2, y2, pc):  
    flag = np.logical_and(pc[:, 0] > x1, pc[:, 0] < x2)
    flag = np.logical_and(flag, pc[:, 1] > y1)
    flag = np.logical_and(flag, pc[:, 1] < y2)
    return flag

In [6]:
def toDinoPrompt(metaData,className):
    listOfParts = metaData[className]
    prompt = ""
    partList = {}
    for i,part in enumerate(listOfParts):
        prompt += f"{className} {part}.".lower()
        partList[f"{className} {part}".lower()] = i
    return prompt,partList

In [5]:
def InferDINOSAMZeroShot(input_pc_file, category, modelDINO, predictorSAM, metaData, device,BOX_TRESHOLD = 0.2,
    TEXT_TRESHOLD = 0.3, SCORE_THRESHOLD=0.2, n_neighbors = 21, n_pass=3, save_dir="tmp"):
    
#     print("-----Zero-shot inference of %s-----" % input_pc_file)
    TEXT_PROMPT,partList = toDinoPrompt(metaData, category)
    
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(f"{save_dir}/rendered_img", exist_ok=True) #create the necessary save directories
    os.makedirs(f"{save_dir}/dino_pred", exist_ok=True)
    os.makedirs(f"{save_dir}/semantic_segDino_KNN", exist_ok=True)
    
    io = IO()
    xyz, rgb = normalize_pc(input_pc_file, save_dir, io, device) #read Point cloud and rgb in the format n,3
    img_dir, pc_idx, screen_coords = render_pc(xyz, rgb, save_dir, device) #create the rendered 2D images and return 
    # pc_idx = hxw where every pixel has a PC index correspondence   
    preds = []
    for i in range(pc_idx.shape[0]):
        image_source, image = load_image(f"{save_dir}/rendered_img/{i}.png") #load rgb images
        predictorSAM.set_image(image_source)
#         print("[dino inference...]")
        boxes, logits, phrases = predict(
                                        model=modelDINO,
                                        image=image,
                                        caption=TEXT_PROMPT,
                                        box_threshold=BOX_TRESHOLD,
                                        text_threshold=TEXT_TRESHOLD
                                    )
        phrases = np.array(phrases) #just to fix indexing

        xyxy = yolobbox2bbox(boxes)*image.shape[-1] #change bbox format to xyxy and scale with image size
        
        nms_mask = []
        for t,bbox in enumerate(xyxy): 
            if check_pc_within_bbox(bbox[0], bbox[1], bbox[2], bbox[3], screen_coords[i]).mean() < 0.95: 
                nms_mask.append(t)
        xyxy = xyxy[nms_mask]
        boxes = boxes[nms_mask]
        logits = logits[nms_mask]
        phrases = phrases[nms_mask]
        
        
        
        nms_indexes = nms(torch.tensor(xyxy) , logits, 0.5).numpy() #non maximum supression

        nms_mask = []
        for t,index in enumerate(nms_indexes):
            if phrases[index].lower() in partList.keys():
                nms_mask.append(t)
        nms_indexes = nms_indexes[nms_mask] #this is a temporary fix for DINO returning different classes that are not in the PROMPT
        # another fix is needed for this as this eleminates some important segments such as chair back as the phrase is not exact to PROMPT

        input_boxes = torch.tensor(xyxy[nms_indexes], device=predictorSAM.device)    
        transformed_boxes = predictorSAM.transform.apply_boxes_torch(input_boxes, image_source.shape[:2])

        if(transformed_boxes.numel() == 0):
            transformed_boxes = None
            
        masks, _, _ = predictorSAM.predict_torch(
            point_coords=None,
            point_labels=None,
            boxes=transformed_boxes,
            multimask_output=False,
        )    #create segmentation masks with sam

        for index,j in enumerate(nms_indexes):
            preds.append({'image_id': i, 'category_id': phrases[j], 
                          'bbox': boxes[j]*image.shape[-1], 
                          'score': logits[j],
                          'mask':masks[index,0]   
                         }
                        )
        annotated_frame = annotate(image_source=image_source, boxes=boxes[nms_indexes], logits=logits[nms_indexes], phrases=phrases[nms_indexes])
        cv2.imwrite(f"{save_dir}/dino_pred/{i}.png", annotated_frame) #save an annotated image for DINO debugging
        
    pc_aggMask = torch.zeros((xyz.shape[0],len(partList)+1)) #this is a segment agg mask we sum all the scores from our bboxes 
    #into their own respective channel, the last channel is for unsegmented parts
    pc_aggMask[:,-1] = SCORE_THRESHOLD #we can set a confidence threshold by setting the unsegmented score
    for prediction in preds:
        maskedPC_idx = pc_idx[prediction["image_id"],prediction["mask"].cpu().numpy()] #this gives you the pc idx of the points that are inside the mask
        index_pcMasked = np.unique(maskedPC_idx)[1:] # we only need the unique idx and the first id is always -1 meaning not found
        pc_aggMask[index_pcMasked,partList[prediction["category_id"]]] += prediction["score"] #add up all the scores for each part
    pc_seg_classes = torch.argmax(pc_aggMask,dim=-1) #select the highest score as our segmentation class
    #if non of the part scores are over the SCORE_THRESHOLD it will be left unsegmented
    partColors = distinctipy.get_colors(len(partList))
    rgb_sem_merged = np.zeros((xyz.shape[0], 3))
    # since projections are not exact meaning not every PC point is rendered into our image our backprojections are not dense
    # use KNN to smooth these backprojections 
    nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree').fit(xyz) #create a knn
    
    results = {"partseg_rgbs":{}}
    for colorId,part in enumerate(partList):
        pc_part_idx = np.zeros((xyz.shape[0]),dtype=int)
        rgb_sem = np.zeros((xyz.shape[0],3))
        pc_part_idx[torch.where(pc_seg_classes==partList[part])] = 1
        
        for pass_ in range(n_pass):
            notColoredIndexes = torch.where(pc_seg_classes!=partList[part]) #find non segmented parts for smoothing

            n_indexes = nn.kneighbors(xyz[notColoredIndexes],n_neighbors+1,return_distance=False)
            n_indexes = n_indexes[:,1:] #get n_neighbors for the points, the first index is always the point itself so delete that
            #we have dense point clouds so distance based measures are not necessary and sometimes give worst results
            flag = pc_part_idx[n_indexes].mean(axis=1) 
            
            flag[np.where(flag>=0.4)] = 1 #and segmnent the points where the mean of neighbours are colored %40 or over
            flag[np.where(flag<0.4)] = 0
            pc_part_idx[notColoredIndexes] = flag
           
        rgb_sem[pc_part_idx.astype(bool)] = partColors[colorId]
        rgb_sem_merged += rgb_sem
        save_colored_pc(f"{save_dir}/semantic_segDino_KNN/{part}.ply", xyz, rgb_sem)
    
        results["partseg_rgbs"][part] = rgb_sem
        
    save_colored_pc(f"{save_dir}/semantic_segDino_KNN/{category}.ply", xyz, rgb_sem_merged)
    results["partList"] = partList
    results["xyz"] = xyz
    
    return results

In [7]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)

In [6]:
modelDINO = load_model("../GroundingDINO/groundingdino/config/GroundingDINO_SwinB_cfg.py",
                       "../GroundingDINO/weights/groundingdino_swinb_cogcoor.pth",
                      device=device
                      )

sam_checkpoint = "../SAM/weights/sam_vit_h_4b8939.pth"
model_type = "vit_h"


sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device="cuda")

predictorSAM = SamPredictor(sam)

final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
metaData = json.load(open("./PartNetE_meta.json"))

In [8]:
category = "Suitcase"
input_pc_file = f"data/{category}//.ply"

In [9]:
category = "Chair"
input_pc_file = f"data/{category}/3091"

In [11]:
import os
import torch
import json
from pytorch3d.io import IO
import numpy as np
from src.utils import normalize_pc
from src.render_pc import render_pc
from src.glip_inference import glip_inference, load_model
from src.gen_superpoint import gen_superpoint
from src.bbox2seg import bbox2seg

def Infer(input_pc_file, category, part_names,glip_demo, save_dir="tmp"):       
#     print("-----Zero-shot inference of %s-----" % input_pc_file)

#     print("[creating tmp dir...]")
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        torch.cuda.set_device(device)
    else:
        device = torch.device("cpu")
    io = IO()
    os.makedirs(save_dir, exist_ok=True)
    
#     print("[normalizing input point cloud...]")
    xyz, rgb = normalize_pc(input_pc_file, save_dir, io, device)
    
#     print("[rendering input point cloud...]")
    img_dir, pc_idx, screen_coords = render_pc(xyz, rgb, save_dir, device)
    
#     print("[glip infrence...]")
    preds = glip_inference(glip_demo, save_dir, part_names)
    
#     print('[generating superpoints...]')
    superpoint = gen_superpoint(xyz, rgb, visualize=True, save_dir=save_dir)
    
#     print('[converting bbox to 3D segmentation...]')
    sem_seg, ins_seg = bbox2seg(xyz, superpoint, preds, screen_coords, pc_idx, part_names, save_dir, solve_instance_seg=False)
    
#     print("[finish!]")
    
    

[nltk_data] Downloading package punkt to
[nltk_data]     /rhome/kamburoglu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /rhome/kamburoglu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
from demo import Infer

In [11]:
from src.glip_inference import load_model_glip

In [13]:
config ="GLIP/configs/glip_Swin_L.yaml"
weight_path = "models/glip_large_model.pth"
#     print("[loading GLIP model...]")
glip_demo = load_model_glip(config, weight_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.pooler.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
def mIOU(label, predictions, case='dino', verbose=True):
    """
    Calculate mean IoU for semantic segmentation predictions.

    Args:
        label: Ground truth label containing 'semantic_seg'.
        predictions: Predictions dictionary from the model.
        case: Specify 'dino' or 'glip' for the respective models.
        verbose: If True, prints detailed IoU calculation logs.

    Returns:
        Tuple of (list of IoUs, mean IoU).
    """
    def log(msg):
        if verbose:
            print(msg)

    try:
        # Validate ground truth (label)
        if 'semantic_seg' not in label.item():
            raise ValueError("The label does not contain 'semantic_seg'.")
        semantic_seg = np.array(label.item()['semantic_seg'])
        unique_parts = np.unique(semantic_seg)
        unique_parts = unique_parts[unique_parts != -1]  # Exclude background (-1)

        ious = []

        if case == 'dino':
            # Validate predictions for 'dino' case
            if 'partList' not in predictions or 'partseg_rgbs' not in predictions:
                raise ValueError(
                    "Invalid 'predictions' format for 'dino'. Expected keys: 'partList' and 'partseg_rgbs'."
                )
            part_list = predictions['partList']
            partseg_rgbs = predictions['partseg_rgbs']

            for part, part_id in part_list.items():
                if part_id not in unique_parts:
                    log(f"{part} not present in ground truth, skipping IoU calculation.")
                    continue

                gt_mask = semantic_seg == part_id
                pred_mask = np.any(partseg_rgbs[part] != [0., 0., 0.], axis=-1)

                intersection = np.logical_and(gt_mask, pred_mask).sum()
                union = np.logical_or(gt_mask, pred_mask).sum()

                iou = intersection / union if union > 0 else 0
                ious.append(iou)
                log(f"IoU for {part}: {iou:.4f}")

        elif case == 'glip':
            # Validate predictions for 'other' case
            if 'part_names_ordered' not in predictions or 'sem_seg' not in predictions:
                raise ValueError(
                    "Invalid 'predictions' format for 'glip'. Expected keys: 'part_names_ordered' and 'sem_seg'."
                )
            part_names = predictions['part_names_ordered']
            sem_seg_pred = np.array(predictions['sem_seg'])
            category = str(predictions['category'])
            
            for idx, part_name in enumerate(part_names):
                if idx not in unique_parts:
                    log(f"{category.lower()} {part_name} not present in ground truth, skipping IoU calculation.")
                    continue

                gt_mask = semantic_seg == idx
                pred_mask = sem_seg_pred == idx

                intersection = np.logical_and(gt_mask, pred_mask).sum()
                union = np.logical_or(gt_mask, pred_mask).sum()

                iou = intersection / union if union > 0 else 0
                ious.append(iou)
                log(f"IoU for {part_name}: {iou:.4f}")

        else:
            # Raise an error for unsupported cases
            raise ValueError(f"Unsupported case: {case}. Use 'dino' or 'glip'.")

        # Calculate mean IoU
        if ious:
            mean_iou = np.mean(ious)
            log(f"Mean IoU: {mean_iou:.4f}")
        else:
            log("No valid parts found in the ground truth. Mean IoU cannot be calculated.")
            mean_iou = None

        return ious, mean_iou

    except Exception as e:
        raise RuntimeError(f"Error calculating IoU: {e}") from e


In [43]:
ious_dino, mean_iou_dino = mIOU(label, predictions=preds, case='dino', verbose=True)
ious_glip, mean_iou_glip = mIOU(label, predictions=anan, case='glip', verbose=True)


chair arm not present in ground truth, skipping IoU calculation.
IoU for chair back: 0.4332
IoU for chair leg: 0.8284
IoU for chair seat: 0.2993
IoU for chair wheel: 0.8238
Mean IoU: 0.5962
chair arm not present in ground truth, skipping IoU calculation.
IoU for back: 0.9348
IoU for leg: 0.8080
IoU for seat: 0.9265
IoU for wheel: 0.8174
Mean IoU: 0.8717


In [13]:
def mIOU_dino_f(label, preds=None, predictions=None, case='dino', verbose=True):
    """
    Calculate mean IoU for semantic segmentation predictions.

    Args:
        label: Ground truth label containing 'semantic_seg'.
        preds: Predictions from the first model (DINO).
        predictions: Predictions from the second model.
        case: Specify 'dino' for the first model or 'other' for the second model.
        verbose: If True, prints detailed IoU calculation logs.

    Returns:
        Tuple of (list of IoUs, mean IoU).
    """
    def log(msg):
        if verbose:
            print(msg)
    try:
        # Validate ground truth (label)
        if 'semantic_seg' not in label.item():
            raise ValueError("The label does not contain 'semantic_seg'.")
        semantic_seg = np.array(label.item()['semantic_seg'])
        unique_parts = np.unique(semantic_seg)
        unique_parts = unique_parts[unique_parts != -1]  # Exclude background (-1)

        ious = []

        if case == 'dino':
            # Validate 'dino' format
            if not preds or 'partList' not in preds or 'partseg_rgbs' not in preds:
                raise ValueError(
                    "Invalid 'preds' format for 'dino' case. Expected keys: 'partList' and 'partseg_rgbs'."
                )
            partList = preds['partList']
            pred_partseg_rgbs = preds["partseg_rgbs"]

            for part, part_id in partList.items():
                if part_id not in unique_parts:
                    log(f"{part} not present in ground truth, skipping IoU calculation.")
                    continue

                gt_mask = semantic_seg == part_id
                pred_mask = np.any(pred_partseg_rgbs[part] != [0., 0., 0.], axis=-1)

                intersection = np.logical_and(gt_mask, pred_mask).sum()
                union = np.logical_or(gt_mask, pred_mask).sum()

                iou = intersection / union if union > 0 else 0
                ious.append(iou)
                log(f"IoU for {part}: {iou:.4f}")

        elif case == 'other':
            # Validate 'other' format
            if not predictions or 'part_names_ordered' not in predictions or 'sem_seg' not in predictions:
                raise ValueError(
                    "Invalid 'predictions' format for 'other' case. Expected keys: 'part_names_ordered' and 'sem_seg'."
                )
            part_names_ordered = predictions['part_names_ordered']
            sem_seg_pred = np.array(predictions['sem_seg'])
            category = str(predictions['category'])

            for idx, part_name in enumerate(part_names_ordered):
                if idx not in unique_parts:
                    log(f"{category.lower()} {part_name} not present in ground truth, skipping IoU calculation.")
                    continue

                gt_mask = semantic_seg == idx
                pred_mask = sem_seg_pred == idx

                intersection = np.logical_and(gt_mask, pred_mask).sum()
                union = np.logical_or(gt_mask, pred_mask).sum()

                iou = intersection / union if union > 0 else 0
                ious.append(iou)
                log(f"IoU for {part_name}: {iou:.4f}")

        else:
            # Raise an error for unsupported cases
            raise ValueError(f"Unsupported case: {case}. Use 'dino' or 'other'.")

        # Calculate mean IoU
        if ious:
            mean_iou = np.mean(ious)
            log(f"Mean IoU: {mean_iou:.4f}")
        else:
            log("No valid parts found in the ground truth. Mean IoU cannot be calculated.")
            mean_iou = None

        return ious, mean_iou

    except Exception as e:
        raise RuntimeError(f"Error calculating IoU: {e}") from e


In [37]:
# First case
ious_dino, mean_iou_dino = mIOU_dino_f(label, preds=preds, case='dino', verbose=True)

# Second case
ious_other, mean_iou_other = mIOU_dino_f(label, predictions=anan, case='other', verbose=True)


chair arm not present in ground truth, skipping IoU calculation.
IoU for chair back: 0.4332
IoU for chair leg: 0.8284
IoU for chair seat: 0.2993
IoU for chair wheel: 0.8238
Mean IoU: 0.5962
Chair arm not present in ground truth, skipping IoU calculation.
IoU for back: 0.9348
IoU for leg: 0.8080
IoU for seat: 0.9265
IoU for wheel: 0.8174
Mean IoU: 0.8717


In [26]:
import numpy as np

# Extract ground truth and predictions for the second model
semantic_seg = np.array(label.item()['semantic_seg'])  # Ground truth
unique_parts = np.unique(semantic_seg)  # Unique parts in the ground truth
unique_parts = unique_parts[unique_parts != -1]  # Exclude -1
sem_seg_pred = np.array(anan['sem_seg'])  # Predictions from the second model
part_names_ordered = anan['part_names_ordered']  # Part names

# Initialize IoU accumulator
ious = []

for idx, part_name in enumerate(part_names_ordered):
    # Check if the part is in the ground truth
    if idx not in unique_parts:
        print(f"{part_name} not present in ground truth, skipping IoU calculation.")
        continue

    # Get the mask for the ground truth
    gt_mask = semantic_seg == idx

    # Get the mask for the predictions
    pred_mask = sem_seg_pred == idx

    # Calculate intersection and union
    intersection = np.logical_and(gt_mask, pred_mask).sum()
    union = np.logical_or(gt_mask, pred_mask).sum()

    # Compute IoU
    iou = intersection / union if union > 0 else 0
    ious.append(iou)

    print(f"IoU for {part_name}: {iou:.4f}")

# Calculate mean IoU only if there are valid IoUs
if ious:
    mean_iou = np.mean(ious)
    print(f"Mean IoU: {mean_iou:.4f}")
else:
    print("No valid parts found in the ground truth. Mean IoU cannot be calculated.")

arm not present in ground truth, skipping IoU calculation.
IoU for back: 0.9524
IoU for leg: 0.8116
IoU for seat: 0.9449
IoU for wheel: 0.8292
Mean IoU: 0.8845


In [14]:
from demoDINO import InferDINOSAMZeroShot

In [15]:
def toDinoPrompt(metaData,className):
    listOfParts = metaData[className]
    prompt = ""
    partList = {}
    for i,part in enumerate(listOfParts):
        prompt += f"{className} {part}.".lower()
        partList[f"{className} {part}".lower()] = i
    return prompt,partList


In [10]:
TEXT_PROMPT,partList = toDinoPrompt(metaData, category)

In [14]:
io = IO()
xyz, rgb = normalize_pc(input_pc_file + "/pc.ply", f'examples/zeroshot_{category}', io, device) #read Point cloud and rgb in the format n,3
img_dir, pc_idx, screen_coords = render_pc(xyz, rgb,  f'examples/zeroshot_{category}', device, "179")

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [20]:
glip_pred = Infer(xyz, rgb, screen_coords, pc_idx, category, metaData[category],glip_demo,device, save_dir="tmp")

In [57]:
modelDINO

GroundingDINO(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): DeformableTransformerEncoderLayer(
          (self_attn): MultiScaleDeformableAttention(
            (sampling_offsets): Linear(in_features=256, out_features=256, bias=True)
            (attention_weights): Linear(in_features=256, out_features=128, bias=True)
            (value_proj): Linear(in_features=256, out_features=256, bias=True)
            (output_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout1): Dropout(p=0.0, inplace=False)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout2): Dropout(p=0.0, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (dropout3): Dropout(p=0.0, inplace=False)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=Tr

In [66]:
category

'Chair'

In [18]:
preds = InferDINOSAMZeroShot(TEXT_PROMPT, partList, xyz, pc_idx, screen_coords, category, 
                                     modelDINO, predictorSAM, device, 
                                     BOX_TRESHOLD=0.2, TEXT_TRESHOLD=0.3, 
                                     SCORE_THRESHOLD=0.2, n_neighbors=21, 
                                     n_pass=5, save_dir=f'examples/zeroshot_{category}')


In [19]:
preds["xyz"].shape

(226129, 3)

In [24]:
label = np.load(input_pc_file + "/label.npy", allow_pickle=True)

In [30]:
label

array({'semantic_seg': array([1, 1, 1, ..., 1, 1, 1], dtype=int32), 'instance_seg': array([0, 0, 0, ..., 0, 0, 0], dtype=int32)},
      dtype=object)